In [1]:

import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
nltk.download('wordnet')
ar = pd.read_csv("Amazon_Consumer_Reviews_of_Amazon_Products.csv")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
#tokenizer = RegexpTokenizer(r'\w+')
#stop = stopwords.words('english')
#stop = tokenizer.tokenize(stop)
#swn
stop = stopwords.words('english')

In [7]:
# review_list_reg = []
# for review in ar['reviews.text']:
#     review_list_reg.append(tokenizer.tokenize(review))

review_list_reg = []
for review in ar['reviews.text']:
     review_list_reg.append(review)

In [8]:
def senti_calc(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    else:
        return (0,0)
    
    for syn in syn_set:
        pos_score += syn.pos_score()
        neg_score += syn.neg_score()
    return (pos_score/len(syn_set), neg_score/len(syn_set))




In [9]:
#list = nltk.word_tokenize(review_list_reg[0])

In [10]:
#list

In [11]:
pos_senti_score = []
neg_senti_score = []
for review in review_list_reg:
    #no_punct = tokenizer.tokenize(review)
    tokens = nltk.word_tokenize(review)
    tags = nltk.pos_tag(tokens)
    length = len(tags)
    pos_score = 0
    neg_score = 0
    for word,tag in tags:
        pos_score += senti_calc(word,tag)[0]
        neg_score += senti_calc(word,tag)[1]
    pos_senti_score.append(pos_score/length)
    neg_senti_score.append(neg_score/length)



In [12]:
#print(pos_senti_score[0])
#print(neg_senti_score[0])

In [13]:
review_rating = []
for rating in ar['reviews.rating']:
     review_rating.append(rating)

In [14]:


#classifier = GaussianNB()
classifier = MultinomialNB()


In [15]:
vectorizer = CountVectorizer(stop_words = stop)
X = vectorizer.fit_transform(review_list_reg)

In [16]:
#vectorizer.get_feature_names_ou
X.shape

(5000, 5275)

In [17]:
#X?

In [18]:
#for x in range(len(pos_senti_score)):
#    np.append(X,pos_senti_score[x],neg_senti_score[
#np.append(X, pos_senti_score, neg_senti_score)
matrix = X.toarray()

In [19]:
pos_senti_score[1]

0.03933974358974359

In [20]:
length = len(pos_senti_score)
for x in range(length):
    pos_senti_score[x] = int(pos_senti_score[x] * 100)

In [21]:
pos_senti_score[0]

4

In [22]:
length = len(neg_senti_score)
for x in range(length):
    neg_senti_score[x] = int(neg_senti_score[x] * 100)

In [23]:
neg_senti_score[0]

5

In [24]:
#matrix?

In [25]:
#sample_arr = np.concatenate((neg_senti_score, pos_senti_score), axis = 0)

In [26]:
#np.hstack((matrix,neg_senti_score))

In [27]:
matrix2 = np.c_[matrix,neg_senti_score]

In [28]:
matrix_f = np.c_[matrix2,pos_senti_score]

In [29]:
matrix_f.shape


(5000, 5277)

In [44]:
matrix_f[4999][5276]

2

In [30]:
#vectorizer.get_params(deep=True)
#Y = vectorizer.inverse_transform(X)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(matrix_f, review_rating)

#X_train, X_test, y_train, y_test = train_test_split(X.toarray(), review_rating)

In [32]:
classifier.fit(X_train, y_train)

MultinomialNB()

In [33]:
y_pred = classifier.predict(X_test)

In [34]:
accuracy = accuracy_score(y_test,y_pred)

In [35]:
accuracy

0.7192

In [36]:
#from sklearn.metrics import accuracy_score,classification_report
#classifier.metrics()

In [37]:
report = classification_report(y_test,y_pred)

C:\Users\JoeKa\anaconda3\envs\cs5050\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
print(report)

              precision    recall  f1-score   support

           1       1.00      0.12      0.21        17
           2       0.00      0.00      0.00        14
           3       0.50      0.07      0.12        42
           4       0.47      0.30      0.37       302
           5       0.77      0.92      0.83       875

    accuracy                           0.72      1250
   macro avg       0.55      0.28      0.31      1250
weighted avg       0.68      0.72      0.68      1250



In [55]:
onestar_class_prob_sorted = classifier.feature_log_prob_[0, :].argsort()[::-1]
twostar_class_prob_sorted = classifier.feature_log_prob_[1, :].argsort()[::-1]
threestar_class_prob_sorted = classifier.feature_log_prob_[2, :].argsort()[::-1]
fourstar_class_prob_sorted = classifier.feature_log_prob_[3, :].argsort()[::-1]
fivestar_class_prob_sorted = classifier.feature_log_prob_[4, :].argsort()[::-1]



In [56]:
print(np.take(vectorizer.get_feature_names(), onestar_class_prob_sorted[2:10]))
print(np.take(vectorizer.get_feature_names(), twostar_class_prob_sorted[2:10]))
print(np.take(vectorizer.get_feature_names(), threestar_class_prob_sorted[2:10]))
print(np.take(vectorizer.get_feature_names(), fourstar_class_prob_sorted[2:10]))
print(np.take(vectorizer.get_feature_names(), fivestar_class_prob_sorted[2:10]))

['amazon' 'bought' 'kindle' 'one' 'week' 'use' 'work' 'tablet']
['use' 'amazon' 'tablet' 'work' 'device' 'bought' 'would' 'apps']
['tablet' 'good' 'great' 'amazon' 'would' 'kids' 'one' 'apps']
['tablet' 'great' 'use' 'good' 'like' 'amazon' 'easy' 'kindle']
['great' 'love' 'tablet' 'use' 'easy' 'bought' 'echo' 'kindle']


print(onestar_class_prob_sorted[0:10])
print(twostar_class_prob_sorted[2:10])
print(threestar_class_prob_sorted[2:10])
print(fourstar_class_prob_sorted[2:10])
print(fivestar_class_prob_sorted[2:10]))

In [58]:
print(onestar_class_prob_sorted[0:10])
print(twostar_class_prob_sorted[0:10]) 
print(threestar_class_prob_sorted[0:10]) 
print(fourstar_class_prob_sorted[0:10]) 
print(fivestar_class_prob_sorted[0:10])


[5275 5276  316  674 2601 3186 5097 4946 5174 4569]
[5275 5276 4946  316 4569 5174 1332  674 5197  394]
[5276 5275 4569 2082 2132  316 5197 2591 3186  394]
[5276 5275 4569 2132 4946 2082 2726  316 1531 2601]
[5276 5275 2132 2814 4569 4946 1531  674 1541 2601]


In [61]:
print(np.take(vectorizer.get_feature_names(), onestar_class_prob_sorted[2:50]))
print(np.take(vectorizer.get_feature_names(), twostar_class_prob_sorted[2:50]))
print(np.take(vectorizer.get_feature_names(), threestar_class_prob_sorted[2:50]))
print(np.take(vectorizer.get_feature_names(), fourstar_class_prob_sorted[2:50]))
print(np.take(vectorizer.get_feature_names(), fivestar_class_prob_sorted[2:50]))


['amazon' 'bought' 'kindle' 'one' 'week' 'use' 'work' 'tablet' 'last'
 'junk' 'get' 'apps' 'buy' 'show' 'first' 'back' 'would' 'useless'
 'device' 'year' 'years' 'purchased' 'product' 'used' 'could' 'less'
 'best' 'constantly' 'video' 'going' 'problem' 'speakers' 'registered'
 'quality' 'fire' 'lasted' 'cloudcam' 'shuts' 'model' 'days' 'charge'
 'tried' 'great' 'also' 'help' 'like' 'echo' 'old']
['use' 'amazon' 'tablet' 'work' 'device' 'bought' 'would' 'apps' 'buy'
 '20' 'returned' 'like' 'quality' 'one' 'nothing' 'ipad' 'time' 'code'
 'better' 'kindle' 'works' 'still' 'generation' 'needs' 'minutes'
 'program' 'ads' 'google' 'screen' 'could' 'every' 'charge' 'set' 'hard'
 'phone' 'games' 'charger' 'slow' 'great' 'battery' 'another' 'connection'
 '12' 'randomly' 'friendly' 'say' 'netflix' 'getting']
['tablet' 'good' 'great' 'amazon' 'would' 'kids' 'one' 'apps' 'screen'
 'ok' 'time' 'echo' 'get' 'bought' 'use' 'works' 'price' 'alexa' 'still'
 'work' 'like' 'books' 'store' 'device' 'ipad'

In [46]:
#classifier.feature_log_prob_[0, :]

array([-8.84303772, -8.84303772, -8.14989054, ..., -8.84303772,
       -3.75544139, -4.0723531 ])

In [47]:
#classifier.feature_log_prob_[1, :]

array([-8.73552519, -8.73552519, -8.73552519, ..., -8.73552519,
       -3.67293015, -4.0441773 ])